In [2]:
import numpy as np

In [3]:
melody_a = [58, 103, 118, 68, 59, 118, 108, 94, 68, 93, 103, 83, 103, 108, 49, 83, 83, 93, 103, 58, 73, 83, 68,
            49, 83, 108, 84, 84, 49, 108, 47, 68, 84, 84, 103, 48, 58, 67, 67, 68, 57, 68, 73, 83, 59, 83, 73, 83,
            48, 49]

melody_b = [38, 103, 118, 68, 29, 118, 108, 94, 68, 93, 103, 83, 103, 108, 49, 83, 83, 93, 103, 58, 73, 83, 68,
            49, 83, 108, 84, 84, 49, 108, 47, 68, 84, 84, 103, 48, 58, 67, 67, 68, 57, 68, 73, 83, 59, 83, 73, 83,
            48, 49]

In [ ]:
def pitch_note_split(melody):
    melody = np.array(melody)
    durations = melody % 5

    melody = (melody - durations) / 5
    durations = 4 / 2 ** durations
    return melody, durations


melody_a, durations_a = pitch_note_split(melody_a)
melody_b, durations_b = pitch_note_split(melody_b)

In [5]:
def transform_relative_pitch(melody):
    # using the formula x = p - avg(p)

    average_pitch = np.average(melody)
    melody = melody - average_pitch
    return melody


relative_melody_a = transform_relative_pitch(melody_a)
relative_melody_b = transform_relative_pitch(melody_b)

In [6]:
def relationship_comparative_line(melody, durations):
    # using the formula tan (a) = (x_b - x_a) / length_x_a

    relationship = (melody[1:] - melody[:-1]) / durations[:-1]
    return relationship


inclination_melody_a = relationship_comparative_line(relative_melody_a, durations_a)
inclination_melody_b = relationship_comparative_line(relative_melody_b, durations_b)

In [7]:
def pitch_difference_limit(relative_melody_1, relative_melody_2):
    """
    LIMIT_x = Max_x - Min_x
    Limit_x : The limit value for the relative pitch difference value
    Max_x : The maximum relative pitch value
    Min_x : The minimum relative pitch value
    Max_x = MAX(Max_x1, Max_x2)
    Minx = MIN(Min_x1, Min_x2)
    Max_x1, Max_x2 : Maximum relative pitch values of melody 1 and melody 2, respectively
    Min_x1, Min_x2 : Minimum relative pitch values of melody 1 and melody 2, respectively
    :param relative_melody_1: main relatively pitched comparative unit (preset)
    :param relative_melody_2: secondary relatively pitched comparative unit (melody to be checked)
    :return: maximum limit of pitch difference
    """

    min_x = min(min(relative_melody_1), min(relative_melody_2))
    max_x = max(max(relative_melody_1), max(relative_melody_2))
    limit = max_x - min_x
    return limit

pitch_diff_limit = pitch_difference_limit(relative_melody_a, relative_melody_b)
pitch_diff_limit

18.0

In [8]:
def inclination_difference_limit(inclination_1, inclination_2):
    """
    Limit_tan(α) = Max_tan(α) – Min_tan(α)
    Limit_tan(α) : The limit value for the inclination
    difference value
    Max_tan(α) : The maximum inclination value
    Min_tan(α) : The minimum inclination value
    Max_tan(α) = MAX(Max_tan(α1), Max_tan(α2))
    Min_tan(α) = MIN(Min_tan(a1), Min_tan(a2))
    Max_tan(α1), Max_tan(α2) : Maximum inclination values of melody 1 and melody 2, respectively
    Min_tan(a1), Min_tan(a2): Minimum inclination values of melody 1 and melody 2, respectively

    :param inclination_1: inclination values of melody 1
    :param inclination_2: inclination values of melody 2
    :return: limit of inclination difference
    """

    min_x = min(min(inclination_1), min(inclination_2))
    max_x = max(max(inclination_1), max(inclination_2))
    limit = max_x - min_x
    return limit

inclination_diff_limit = inclination_difference_limit(inclination_melody_a, inclination_melody_b)

In [9]:
def pitch_similarity(x1, x2, pitch_diff_lim):
    """
    Formula: 1 -  | x1 = x2 | / Limit_x

    :param x1: relative pitch values of the preset
    :param x2: relative pitch values of the melody being compared
    :param pitch_diff_lim: maximum limit of pitch difference
    :return: pitch similarity score
    """
    return 1 - abs(x1 - x2) ** 0.5 / pitch_diff_lim ** 0.5

pitch_similarity_result =  pitch_similarity(relative_melody_a, relative_melody_b, pitch_diff_limit)
pitch_similarity_result

array([0.54053171, 0.89459074, 0.89459074, 0.89459074, 0.43235379,
       0.89459074, 0.89459074, 0.89459074, 0.89459074, 0.89459074,
       0.89459074, 0.89459074, 0.89459074, 0.89459074, 0.89459074,
       0.89459074, 0.89459074, 0.89459074, 0.89459074, 0.89459074,
       0.89459074, 0.89459074, 0.89459074, 0.89459074, 0.89459074,
       0.89459074, 0.89459074, 0.89459074, 0.89459074, 0.89459074,
       0.89459074, 0.89459074, 0.89459074, 0.89459074, 0.89459074,
       0.89459074, 0.89459074, 0.89459074, 0.89459074, 0.89459074,
       0.89459074, 0.89459074, 0.89459074, 0.89459074, 0.89459074,
       0.89459074, 0.89459074, 0.89459074, 0.89459074, 0.89459074])

In [10]:
def inclination_similarity(inclination_1, inclination_2, inclination_diff_lim):
    """
    Formula: 1 - | tan(a1) - tan(a2) | / Limit_tan(a)
    :param inclination_1: inclination values of the preset
    :param inclination_2: inclination values of the melody to be compared
    :param inclination_diff_lim: maximum limit of inclination difference
    :return: inclination similarity score
    """
    return 1 - abs(inclination_1 - inclination_2) ** 0.5 / inclination_diff_lim ** 0.5

inclination_similarity_result = inclination_similarity(inclination_melody_a, inclination_melody_b, inclination_diff_limit)
inclination_similarity_result

array([0.71715729, 1.        , 1.        , 0.65358984, 0.51010205,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        ])

In [11]:

def weighted_pitch_similarity(pitch_sim_result, length):
    """
    Calculates the weighted average of pitch similarity scores having the note duration as weights
    """
    return np.sum(pitch_sim_result * length) / np.sum(length)

weighted_pitch_similarity_result = weighted_pitch_similarity(pitch_similarity_result, durations_a)

In [12]:
def weighted_inclination_similarity(inclination_sim_result, length):
    """
    Calculates the weighted average of pitch similarity scores having the note duration as weights
    """
    return np.sum(inclination_sim_result * length[:-1]) / np.sum(length[:-1])

weighted_inclination_similarity_result = weighted_inclination_similarity(inclination_similarity_result, durations_a)

In [13]:
def overall_similarity(weighted_pitch_sim_result, weighted_inclination_sim_result):
    return (weighted_pitch_sim_result + weighted_inclination_sim_result) / 2


similarity_percentage = overall_similarity(weighted_inclination_similarity_result, weighted_inclination_similarity_result)
similarity_percentage * 100

98.17874614944436